In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Subset
from transformers import AutoModel, AutoTokenizer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict

## 2. データセットの準備とクライアントへの分割

In [14]:
# データセットの準備とクライアントへの分割 (変更)
# 1クライアント1ユーザーをシミュレート
num_users = 100
num_items = 50
num_clients = num_users # クライアント数とユーザー数を同じにする

user_texts = {i: f"This user likes movies about {i % 5} and enjoys {i % 3}." for i in range(num_users)}

interactions_list = []
for u_id in range(num_users):
    for i_id in range(num_items):
        if np.random.rand() > 0.7:
            interactions_list.append([u_id, i_id, 1])
        else:
            interactions_list.append([u_id, i_id, 0])

interactions = torch.tensor(interactions_list, dtype=torch.float32)

client_user_map = {} # {client_id: user_id}
client_datasets = {}
for u_id in range(num_users):
    client_id = u_id # クライアントIDをユーザーIDと同じにする NOTE: 1クライアント複数ユーザのようなケースにも対応できるようにしているが今回は、1クライアント1ユーザ.
    client_user_map[client_id] = u_id

    # 各クライアントはそのユーザーのインタラクションデータのみを持つ
    client_interactions_indices = [item for item, (user, _, _) in enumerate(interactions_list) if user == u_id]
    client_subset = Subset(
        TensorDataset(interactions[:, 0].long(), interactions[:, 1].long(), interactions[:, 2]),
        client_interactions_indices
    )
    client_datasets[client_id] = DataLoader(client_subset, batch_size=10, shuffle=True)



## 3. 軽量 LLM 埋め込みモデルのロード

In [15]:
# 軽量 LLM 埋め込みモデルのロード (変更なし)
plm_model_name = "sentence-transformers/all-MiniLM-L6-v2"
plm_tokenizer = AutoTokenizer.from_pretrained(plm_model_name)
plm_model = AutoModel.from_pretrained(plm_model_name)

# PLMは学習済みモデルのため、勾配計算を無効化
for param in plm_model.parameters():
    param.requires_grad = False

plm_embedding_dim = plm_model.config.hidden_size
print(f"PLM embedding dimension: {plm_embedding_dim}")

PLM embedding dimension: 384


## 4. モデルの定義 (変更あり: グローバルアイテム埋め込みの追加)
各クライアントが持つモデルに加えて、サーバーが管理する「グローバルアイテム埋め込み」を導入します。これは、論文における 

theta_item に相当します 。

In [16]:
class ClientModel(nn.Module):
    def __init__(
        self,
        num_items,
        item_embedding_dim,
        plm_model,
        plm_embedding_dim,
        joint_embedding_output_dim
    ):
        super(ClientModel, self).__init__()
        self.plm_model = plm_model

        # Joint Embedding Layer の線形変換部分 (ユーザーのローカルパラメータ)
        # 論文の「trainable linear layer」[cite: 75]
        self.user_joint_embedding_linear = nn.Linear(plm_embedding_dim, joint_embedding_output_dim)

        # アイテム埋め込み (各クライアントのローカルパラメータとして初期化される)
        self.local_item_embedding = nn.Embedding(num_items, item_embedding_dim)

        # 予測層. 本来は、ここはMLPで3層になる予定だが、今回は、1層の単純線形層で出力する.
        self.prediction_layer = nn.Linear(joint_embedding_output_dim + item_embedding_dim, 1)

    def forward(self, item_ids, user_texts_batch):
        # ユーザーのテキスト特徴をPLMで埋め込み
        encoded_input = plm_tokenizer(user_texts_batch, padding=True, truncation=True, return_tensors='pt')
        plm_output = self.plm_model(**encoded_input).last_hidden_state[:, 0, :] # [CLS]トークンの埋め込みを使用 [cite: 73]

        # Joint Embedding Layer の線形変換
        # 論文の式(3): e_u = h(v_u) = v_u W_d1xd + b [cite: 78]
        user_embedding = self.user_joint_embedding_linear(plm_output)

        # ローカルのアイテム埋め込み
        item_embedding = self.local_item_embedding(item_ids)

        # ユーザー埋め込みとアイテム埋め込みを結合
        combined_features = torch.cat((user_embedding, item_embedding), dim=1)

        # 予測
        prediction = torch.sigmoid(self.prediction_layer(combined_features))
        # 論文では「user joint embedding weights」[cite: 61] と「local item embeddings」[cite: 61] をサーバーに送信するとある。
        # ここでは、user_joint_embedding_linear.weight と local_item_embedding.weight を返す
        return prediction, self.user_joint_embedding_linear.weight, self.local_item_embedding.weight

In [ ]:
class Server:
    def __init__(
        self,
        num_users,
        num_items,
        item_embedding_dim,
        joint_embedding_output_dim,
    ):
        # グローバルアイテム埋め込みは、全アイテムの埋め込み行列として管理
        self.global_item_embedding = nn.Embedding(num_items, item_embedding_dim)
        self.num_users = num_users
        self.num_items = num_items
        self.item_embedding_dim = item_embedding_dim
        self.joint_embedding_output_dim = joint_embedding_output_dim

    def build_user_relationship_graph(
        self,
        user_linear_weights_map,
    ):
        """
        各ユーザーのuser_joint_embedding_linear.weightからユーザー関係グラフを構築します。
        論文の式 (15) に基づいています。

        Args:
            user_linear_weights_map (dict): {user_id: user_joint_embedding_linear.weight.data (flattened)}

        Returns:
            np.ndarray: ユーザーグラフの隣接行列 (NumPy配列)
            list: グラフのノード順に対応するユーザーIDのリスト
        """
        # ユーザIDをグラフのノードとして扱う
        sorted_user_ids = sorted(user_linear_weights_map.keys())

        # 各ユーザの線形層の重みベクトルを収集する.
        # 論文の「w_i = vec(W_i)」に相当 [cite: 105]
        # FIXME: 今は、クライアント側でやってしまっているが、行列をベクトル化するのは、サーバ側でやっておきたい.
        user_weight_vectors = np.array([
            user_linear_weights_map[u_id].cpu().numpy() for u_id in sorted_user_ids
        ])

        # cos類似度で類似度行列を計算(S_ij)
        # 論文の式(15)に相当 [cite: 106]
        similarity_matrix = cosine_similarity(user_weight_vectors)

        # 各ユーザーの上位N個の類似ユーザーを選択してグラフを構築 (ここでは簡単のため、全てのユーザー間の類似度を使用)
        # 論文のStep2では「top-N in the highest similarity list」とあるが [cite: 108]、
        # ここでは完全な類似度グラフ (隣接行列) を使用。
        # 厳密には、ここで閾値を設けるか、上位K個のみを選択して疎なグラフを構築すべき。
        # S' に相当 [cite: 108]
        user_graph_adj = similarity_matrix 

        return user_graph_adj, sorted_user_ids

    def aggregate_item_embeddings(
        self,
        user_local_item_weights,
        user_graph_adj,
        sorted_user_ids,
    ):
        """
        ユーザー関係グラフに基づいて、アイテム埋め込みをグローバルに集約します。
        論文の式 (16) と (17) に基づいています。

        Args:
            user_local_item_weights (dict): {user_id: local_item_embedding.weight.data (Tensor)}
            user_graph_adj (np.ndarray): ユーザーグラフの隣接行列
            sorted_user_ids (list): user_graph_adj のノード順に対応するユーザーIDのリスト

        Returns:
            torch.Tensor: 更新されたグローバルアイテム埋め込みの重み
        """
        # グラフの順序に合わせて各ユーザーのアイテム埋め込みを行列Aとしてまとめる
        # A は (num_users, num_items, item_embedding_dim)
        # 論文の「A is the round item embedding matrix, the I-th row represents the item embedding obtained from user i」に相当 
        item_embedding_matrix_A = torch.stack([
            user_local_item_weights[u_id] for u_id in sorted_user_ids
        ]) # (num_users, num_items, item_embedding_dim)

        # グラフの正規化 (S'')
        # NOTE: 正規化が必要な理由
        ### スケール調整と数値安定性: 隣接行列をそのまま使うと、ノードの次数（接続数）が大きいほど、そのノードから受け取る情報の合計が非常に大きくなってしまいます。これにより、特徴量のスケールが大きくなりすぎたり、訓練中に勾配爆発を引き起こしたりする可能性があります。正規化は、この影響を均一化し、数値的な安定性を確保するのに役立ちます 。
        ### 特徴量の平滑化と拡散: 正規化は、ノードの特徴量（この場合はアイテム埋め込み）が隣接ノードに適切に伝播・拡散されることを保証します。正規化されていない場合、高次数のノードが支配的になり、低次数のノードの情報が埋もれてしまう可能性があります。
        ### GCNの理論的根拠: LightGCNのようなGCNモデルでは、グラフ畳み込み操作がグラフ上の情報の平滑化と拡散として機能します 。正規化は、この平滑化プロセスが効果的に機能するために不可欠です。
        row_sums_graph = np.sum(user_graph_adj, axis=1, keepdims=True)
        row_sums_graph[row_sums_graph == 0] = 1 # ゼロ除算回避
        normalized_user_graph_adj = user_graph_adj / row_sums_graph
        normalized_user_graph_adj_tensor = torch.tensor(normalized_user_graph_adj, dtype=torch.float32)

        # グラフ畳み込み (R = S'' A)
        # R は (num_users, num_items, item_embedding_dim) となる
        # MatMul: (num_users, num_users) x (num_users, num_items, item_embedding_dim)
        # Einstein Summation Convention: 'ij, jkd -> ikd'
        R_tensor = torch.einsum('ij, jkd -> ikd', normalized_user_graph_adj_tensor, item_embedding_matrix_A)

        # グローバルアイテム埋め込みの更新 (θ_global = DR)
        # 論文の式(17)に相当 [cite: 113]
        # ここではDを全ユーザーの単純平均と解釈 (Rの0次元目を平均)
        new_global_item_embedding_weight = R_tensor.mean(dim=0) # (num_items, item_embedding_dim)

        # サーバーのグローバルアイテム埋め込みを直接更新
        self.global_item_embedding.weight.data.copy_(new_global_item_embedding_weight)

        return self.global_item_embedding.weight.data

## 5. 学習ループの変更 (フェデレーテッド学習のシミュレーション)
この学習ループでは、以下のステップをシミュレートします。

グローバル配布: サーバーがグローバルアイテム埋め込みを各クライアントに配布します。

ローカル学習: 各クライアントは自身のデータを使ってモデルを学習し、ユーザーのジョイント埋め込み層の重みと、ローカルで更新されたアイテム埋め込み（のテンソル）をサーバーにアップロードします。

グラフ集約: サーバーはアップロードされたユーザー埋め込み層の重みからユーザー関係グラフを構築し、それに基づいてグローバルアイテム埋め込みを集約・更新します。

In [18]:
# モデルのハイパーパラメータ
item_embedding_dim = 32
joint_embedding_output_dim = 32 

# サーバーのインスタンス化
server = Server(num_users, num_items, item_embedding_dim, joint_embedding_output_dim)

# 各クライアントのモデルを辞書で保持
client_models = {}
client_optimizers = {}
for client_id in range(num_clients):
    client_models[client_id] = ClientModel(
        num_items,
        item_embedding_dim,
        plm_model,
        plm_embedding_dim,
        joint_embedding_output_dim
    )
    # NOTE:
    # クライアントごとに最適化するパラメータを設定
    # ここでは、user_joint_embedding_linear, local_item_embedding, prediction_layer が対象
    # 単純にoptim.Adam(params = client_models[client_id].parameters(), lr=0.001)とすると、
    # PLMも学習可能パラメータとなってしまうので、
    # PLMのパラメータを除外したパラメータのみを取得してから、設定する.
    trainable_params = [
        p for name, p in client_models[client_id].named_parameters()
        if not name.startswith('plm_model.')
    ]

    client_optimizers[client_id] = optim.Adam(
        params=trainable_params,
        lr=0.001
    )


# 学習ループ (フェデレーテッド学習ラウンド)
num_communication_rounds = 10
local_epochs = 1 



for round_num in range(num_communication_rounds):
    print(f"\n--- Communication Round {round_num + 1}/{num_communication_rounds} ---")

    # サーバーからグローバルアイテム埋め込みをクライアントに配布
    # 論文のステップ「Global Distribution: Updated global project embeddings are broadcast to all clients for next-round initialization.」[cite: 63]
    for client_id in range(num_clients):
        client_models[client_id].local_item_embedding.weight.data.copy_(server.global_item_embedding.weight.data)

    user_linear_weights_for_graph = {} # {user_id: user_joint_embedding_linear.weight.data (flattened)}
    user_local_item_weights_to_server = {} # {user_id: local_item_embedding.weight.data (Tensor)}

    # クライアントのローカル学習
    # 論文のステップ「Local Training」[cite: 60]
    for client_id in range(num_clients):
        model = client_models[client_id]
        optimizer = client_optimizers[client_id]
        dataloader = client_datasets[client_id]

        model.train()
        local_loss = 0

        # このクライアントが担当するユーザーID
        current_user_id = client_user_map[client_id] 

        for epoch in range(local_epochs):
            for user_ids_batch, item_ids_batch, labels_batch in dataloader: 
                current_user_texts = [user_texts[uid.item()] for uid in user_ids_batch]

                optimizer.zero_grad()

                predictions, user_joint_embedding_linear_weight, local_item_embedding_weight = model(
                    item_ids_batch, current_user_texts
                )

                # 損失計算 (L_1のみ)
                # 論文の式(10) L_1(y,y^) [cite: 91]
                loss = nn.BCELoss()(predictions.squeeze(), labels_batch)

                loss.backward()
                optimizer.step()
                local_loss += loss.item()

        # クライアントがサーバーにアップロードするパラメータを収集
        # 論文のステップ「Parameter Uploading: Clients transmit user joint embedding weights and local item embeddings to the server.」[cite: 61]
        # 各ユーザー（クライアント）は自身の user_joint_embedding_linear.weight をベクトル化してアップロード
        user_linear_weights_for_graph[current_user_id] = user_joint_embedding_linear_weight.data.clone().flatten()

        # 各ユーザー（クライアント）は自身の local_item_embedding.weight もアップロード
        user_local_item_weights_to_server[current_user_id] = local_item_embedding_weight.data.clone()

        print(f"  Client {client_id} (User {current_user_id}) local loss: {local_loss / len(dataloader):.4f}")

    # サーバーでの処理
    # 論文のステップ「Graph Aggregation: The server constructs user relation graphs from text embeddings and aggregates parameters through graph convolution.」[cite: 62]
    # ユーザー関係グラフの構築
    # 論文のステップ「Build User Relaction Graph」[cite: 102, 122]
    user_graph_adj, sorted_user_ids_for_graph = server.build_user_relationship_graph(
        user_linear_weights_for_graph
    )

    # アイテム埋め込みの集約
    # 論文のステップ「Learn user common item embeddings with Eq.(16)」[cite: 122] と
    # 「Learn globally shared item embedding θ_global with Eq.(17)」[cite: 122]
    server.aggregate_item_embeddings(
        user_local_item_weights_to_server,
        user_graph_adj,
        sorted_user_ids_for_graph
    )

    print(f"Round {round_num + 1} completed. Global item embeddings updated.")

print("Federated training completed.")


--- Communication Round 1/10 ---
  Client 0 (User 0) local loss: 0.6512
  Client 1 (User 1) local loss: 0.7042
  Client 2 (User 2) local loss: 0.7052
  Client 3 (User 3) local loss: 0.7133
  Client 4 (User 4) local loss: 0.6571
  Client 5 (User 5) local loss: 0.6896
  Client 6 (User 6) local loss: 0.6302
  Client 7 (User 7) local loss: 0.6649
  Client 8 (User 8) local loss: 0.6646
  Client 9 (User 9) local loss: 0.6635
  Client 10 (User 10) local loss: 0.7050
  Client 11 (User 11) local loss: 0.5996
  Client 12 (User 12) local loss: 0.6631
  Client 13 (User 13) local loss: 0.6504
  Client 14 (User 14) local loss: 0.7252
  Client 15 (User 15) local loss: 0.6619
  Client 16 (User 16) local loss: 0.6855
  Client 17 (User 17) local loss: 0.6503
  Client 18 (User 18) local loss: 0.6791
  Client 19 (User 19) local loss: 0.6758
  Client 20 (User 20) local loss: 0.6300
  Client 21 (User 21) local loss: 0.6566
  Client 22 (User 22) local loss: 0.6327
  Client 23 (User 23) local loss: 0.7275
  